In [8]:
from dataloaders import data_set,data_dict
import torch
import yaml
import os
import torch.nn as nn

import torch
import torch.optim as optim

from dataloaders import data_set,data_dict
from models.model_builder import model_builder


import numpy as np

import ray
from ray import tune, air
from ray.air import session, Checkpoint
from ray.tune.examples.mnist_pytorch import ConvNet, get_data_loaders, train, test
from ray.tune.schedulers import PopulationBasedTraining

from functionapi import train_net
from PBTTrainable import RayModel

from analysis import plot_hp_history, get_average_frame
import random

from utils import adjust_learning_rate_class
from ray.tune.schedulers.pb2 import PB2

from pbtexperiment import validation, _get_data

In [12]:
experiment_path = "/home/janis/PopulationBasedTraining/RayTuneResults/hapt/pbt/RayModel_2023-08-18_21-01-26"

restored_tuner = tune.Tuner.restore(experiment_path,trainable=RayModel)
results_grid = restored_tuner.get_results()
best_result = results_grid.get_best_result(mode="max", metric="mean_accuracy")

best_result.metrics_dataframe

get_average_frame(results_grid, "mixup_probability").plot(title="Average Mixup Probability over Time")
get_average_frame(results_grid, "random_augmentation_prob").plot(title="Average Randomaug Probability over Time")


best_result.metrics_dataframe.plot("training_iteration", "mean_accuracy", title="Mean Accuracy of best over time")
best_result.metrics_dataframe.plot("training_iteration", "random_augmentation_prob", title="Random Aug Prop of best over time")
best_result.metrics_dataframe.plot("training_iteration", "mixup_probability", title="Mixup Prop of best over time")


#results_grid[10].metrics_dataframe.plot("training_iteration", "mean_accuracy")
#results_grid[10].metrics_dataframe.plot("training_iteration", "mixup_probability")

plot_hp_history(results_grid, "mixup_probability")
plot_hp_history(results_grid, "random_augmentation_prob")
plot_hp_history(results_grid, "random_aug_first")

#best_result.metrics_dataframe

best_checkpoint = best_result.checkpoint
checkpoint_dict = best_checkpoint.to_dict()

2023-08-18 21:12:42,179	INFO experiment_analysis.py:972 -- No trial data passed in during `ExperimentAnalysis` initialization -- you are most likely loading the experiment after it has completed.
Loading trial data from the experiment checkpoint file. This may result in loading some stale information, since checkpointing is periodic.


,mean_accuracy,total_loss,done,training_iteration,trial_id,date,timestamp,time_this_iter_s,time_total_s,pid,hostname,node_ip,time_since_restore,iterations_since_restore
0,0.530086,1.626942,False,1,a2ce6_00004,2023-08-18_21-01-39,1692385299,8.309813,8.309813,88219,janis-B660M-GAMING-X-AX-DDR4,10.12.179.67,8.309813,1
1,0.731614,0.996945,False,2,a2ce6_00004,2023-08-18_21-01-45,1692385305,6.144265,14.454078,88219,janis-B660M-GAMING-X-AX-DDR4,10.12.179.67,14.454078,2
2,0.740210,0.835645,False,3,a2ce6_00004,2023-08-18_21-01-51,1692385311,6.167378,20.621456,88219,janis-B660M-GAMING-X-AX-DDR4,10.12.179.67,20.621456,3
3,0.772684,0.734828,False,4,a2ce6_00004,2023-08-18_21-01-57,1692385317,6.146593,26.768049,88219,janis-B660M-GAMING-X-AX-DDR4,10.12.179.67,26.768049,4
4,0.790831,0.632714,False,5,a2ce6_00004,2023-08-18_21-02-04,1692385324,6.113481,32.881530,88219,janis-B660M-GAMING-X-AX-DDR4,10.12.179.67,32.881530,5
5,0.895893,0.268495,False,6,a2ce6_00004,2023-08-18_21-03-53,1692385433,12.012883,41.268545,91219,janis-B660M-GAMING-X-AX-DDR4,10.12.179.67,12.012883,1
6,0.902579,0.265734,False,7,a2ce6_00004,2023-08-18_21-03-59,1692385439,5.858447,47.126992,91219,janis-B660M-GAMING-X-AX-DDR4,10.12.179.67,17.871330,2
7,0.908309,0.245502,False,8,a2ce6_00004,2023-08-18_21-04-05,1692385445,5.913243,53.040235,91219,janis-B660M-GAMING-X-AX-DDR4,10.12.179.67,23.784572,3
8,0.903534,0.248780,False,9,a2ce6_00004,2023-08-18_21-04-11,1692385451,6.161564,59.201798,91219,janis-B660M-GAMING-X-AX-DDR4,10.12.179.67,29.946136,4
9,0.913085,0.235555,False,10,a2ce6_00004,2023-08-18_21-04-17,1692385457,6.102260,65.304058,91219,janis-B660M-GAMING-X-AX-DDR4,10.12.179.67,36.048396,5


In [ ]:
experiment_folder = "/home/janis/PopulationBasedTraining/RayTuneResults/hapt/pbt"

def compare_experiments(experiment_folder, trial, parameter):
    ax=None

    for f in os.listdir(experiment_folder):
        restored_tuner = tune.Tuner.restore(experiment_folder+"/"+f,trainable=pb.train_net)
        grid = restored_tuner.get_results()
        
        if trial == "avg":
            frame = pb.get_average_frame(grid, parameter)
            ax = frame.plot(ax=ax, title="Avg " + parameter + " over time")
        elif trial == "best":
            frame = grid.get_best_result(mode="max", metric="mean_accuracy").metrics_dataframe[["training_iteration", parameter]]
            ax = frame.plot(ax=ax, x="training_iteration",y=parameter, title="Best Trial " + parameter + " over time")

        
#compare_experiments(experiment_folder, "best", "mixup_probability")

compare_experiments(experiment_folder, "avg", "mixup_probability")

#compare_experiments(experiment_folder, "best", "random_augmentation_prob")

compare_experiments(experiment_folder, "best", "mean_accuracy")


2023-08-16 21:46:34,536	INFO experiment_analysis.py:972 -- No trial data passed in during `ExperimentAnalysis` initialization -- you are most likely loading the experiment after it has completed.
Loading trial data from the experiment checkpoint file. This may result in loading some stale information, since checkpointing is periodic.
2023-08-16 21:46:34,546	WARNING experiment_analysis.py:916 -- Failed to read the results for 15 trials:
- /home/janis/PopulationBasedTraining/RayTuneResults/hapt/pbt/train_net_2023-08-14_17-17-21/train_net_aa324_00000_0_mixup_probability=0.7000,random_aug_first=False,random_augmentation_prob=0.9000_2023-08-14_17-17-21
- /home/janis/PopulationBasedTraining/RayTuneResults/hapt/pbt/train_net_2023-08-14_17-17-21/train_net_aa324_00001_1_mixup_probability=1.0000,random_aug_first=True,random_augmentation_prob=0.1000_2023-08-14_17-17-21
- /home/janis/PopulationBasedTraining/RayTuneResults/hapt/pbt/train_net_2023-08-14_17-17-21/train_net_aa324_00002_2_mixup_probabi

TypeError: 'NoneType' object is not subscriptable

In [ ]:
experiment_folder = "/home/janis/PopulationBasedTraining/RayTuneResults/hapt/pbt"

for f in os.listdir(experiment_folder):
    restored_tuner = tune.Tuner.restore(experiment_folder+"/"+f,trainable=pb.train_net)
    grid = restored_tuner.get_results()
    best = grid.get_best_result(mode="max", metric="mean_accuracy")
    print("Best Trial Mean Accuracy: {}, iterations: {}, mixup Propability: {}, Random Aug Propability: {}".
          format(best.metrics["mean_accuracy"], best.metrics["training_iteration"], 
                 best.metrics["mixup_probability"], best.metrics["random_augmentation_prob"]))

2023-08-14 17:14:35,991	INFO experiment_analysis.py:972 -- No trial data passed in during `ExperimentAnalysis` initialization -- you are most likely loading the experiment after it has completed.
Loading trial data from the experiment checkpoint file. This may result in loading some stale information, since checkpointing is periodic.
2023-08-14 17:14:36,020	INFO experiment_analysis.py:972 -- No trial data passed in during `ExperimentAnalysis` initialization -- you are most likely loading the experiment after it has completed.
Loading trial data from the experiment checkpoint file. This may result in loading some stale information, since checkpointing is periodic.
2023-08-14 17:14:36,049	INFO experiment_analysis.py:972 -- No trial data passed in during `ExperimentAnalysis` initialization -- you are most likely loading the experiment after it has completed.
Loading trial data from the experiment checkpoint file. This may result in loading some stale information, since checkpointing is pe

Best Trial Mean Accuracy: 0.9413184492381959, iterations: 100, mixup Propability: 0.0, Random Aug Propability: 1.0
Best Trial Mean Accuracy: 0.9321164579876301, iterations: 55, mixup Propability: 0.7000000000000001, Random Aug Propability: 0.4
Best Trial Mean Accuracy: 0.921255091265651, iterations: 18, mixup Propability: 0.4, Random Aug Propability: 0.4
Best Trial Mean Accuracy: 0.9413184492381959, iterations: 100, mixup Propability: 0.2, Random Aug Propability: 0.30000000000000004
